In [1]:
# =============================================================================
# BLOCK 1: SETUP, IMPORTS, AND DATA LOADING
# =============================================================================
import warnings
warnings.filterwarnings('ignore')
import time

# --- Library Imports ---
import pandas as pd
import numpy as np
import gc
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import optuna

print("Libraries imported successfully.")

# --- Helper Function for Winkler Score ---
def winkler_score(y_true, lower, upper, alpha=0.1, return_coverage=False):
    width = upper - lower
    penalty_lower = np.where(y_true < lower, (2 / alpha) * (lower - y_true), 0)
    penalty_upper = np.where(y_true > upper, (2 / alpha) * (y_true - upper), 0)
    score = width + penalty_lower + penalty_upper
    if return_coverage:
        coverage = np.mean((y_true >= lower) & (y_true <= upper))
        return np.mean(score), coverage
    return np.mean(score)

# --- Global Constants ---
N_SPLITS = 5
RANDOM_STATE = 42
DATA_PATH = './'
N_OPTUNA_TRIALS = 30 # A strong number for a comprehensive search
COMPETITION_ALPHA = 0.1

# --- Load Raw Data ---
try:
    # We drop the low-variance columns they identified right away
    drop_cols=['id', 'golf', 'view_rainier', 'view_skyline', 'view_lakesamm', 'view_otherwater', 'view_other']
    df_train = pd.read_csv(DATA_PATH + 'dataset.csv').drop(columns=drop_cols)
    df_test = pd.read_csv(DATA_PATH + 'test.csv').drop(columns=drop_cols)
    print("Raw data loaded successfully.")
except FileNotFoundError:
    print("ERROR: Could not find 'dataset.csv' or 'test.csv'.")
    exit()

# --- Prepare Target Variable ---
y_true = df_train['sale_price'].copy()
# The mean-error model works best when predicting the raw price directly
# So, we will NOT log-transform the target this time.
# df_train.drop('sale_price', axis=1, inplace=True) # We keep sale_price for FE

print("Setup complete.")

Libraries imported successfully.
Raw data loaded successfully.
Setup complete.


In [2]:
# =============================================================================
# BLOCK 2: SYNTHESIZED FEATURE ENGINEERING (CORRECTED)
# =============================================================================
print("--- Starting Block 2: Synthesized Feature Engineering ---")

def create_synthesized_features(df_train, df_test):
    # Combine for consistent processing and reset the index
    df_train['is_train'] = 1
    df_test['is_train'] = 0
    # Store the original id for later, as reset_index will remove it
    train_ids = df_train.index
    test_ids = df_test.index
    all_data = pd.concat([df_train, df_test], axis=0).reset_index(drop=True)

    # --- A) Brute-Force Numerical Interactions ---
    print("Creating brute-force numerical interaction features...")
    NUMS = ['area', 'land_val', 'imp_val', 'sqft_lot', 'sqft', 'sqft_1', 'grade', 'year_built']
    for i in range(len(NUMS)):
        for j in range(i + 1, len(NUMS)):
            all_data[f'{NUMS[i]}_x_{NUMS[j]}'] = all_data[NUMS[i]] * all_data[NUMS[j]]

    # --- B) Date Features ---
    all_data['sale_date'] = pd.to_datetime(all_data['sale_date'])
    all_data['year'] = all_data['sale_date'].dt.year
    all_data['month'] = all_data['sale_date'].dt.month
    all_data['year_diff'] = all_data['year'] - all_data['year_built']

    # --- C) TF-IDF Text Features ---
    print("Creating TF-IDF features for text columns...")
    text_cols = ['subdivision', 'zoning', 'city', 'sale_warning', 'join_status', 'submarket']
    all_data[text_cols] = all_data[text_cols].fillna('missing').astype(str)

    for col in text_cols:
        tfidf = TfidfVectorizer(analyzer='char', ngram_range=(3, 5), max_features=128, binary=True)
        tfidf_matrix = tfidf.fit_transform(all_data[col])
        svd = TruncatedSVD(n_components=8, random_state=RANDOM_STATE)
        tfidf_svd = svd.fit_transform(tfidf_matrix)
        tfidf_df = pd.DataFrame(tfidf_svd, columns=[f'{col}_tfidf_svd_{i}' for i in range(8)])
        # This concat will now work because both have a simple 0-based index
        all_data = pd.concat([all_data, tfidf_df], axis=1)

    # --- D) Log transform some of the new interaction features ---
    for c in ['land_val_x_imp_val', 'land_val_x_sqft', 'imp_val_x_sqft']:
        if c in all_data.columns:
            # Add a small constant to avoid log(0)
            all_data[c] = np.log1p(all_data[c].fillna(0))
            
    # --- E) Final Cleanup ---
    print("Finalizing feature set...")
    cols_to_drop = ['sale_date', 'subdivision', 'zoning', 'city', 'sale_warning', 'join_status', 'submarket']
    all_data = all_data.drop(columns=cols_to_drop)
    all_data.fillna(0, inplace=True)

    # Separate final datasets
    X = all_data[all_data['is_train'] == 1].drop(columns=['is_train', 'sale_price'])
    X_test = all_data[all_data['is_train'] == 0].drop(columns=['is_train', 'sale_price'])
    
    # Restore the original 'id' as the index
    X.index = train_ids
    X_test.index = test_ids
    
    X_test = X_test[X.columns]
    
    return X, X_test

# We need to re-run this from the original dataframes
X, X_test = create_synthesized_features(df_train, df_test)

print(f"\nSynthesized FE complete. Total features: {X.shape[1]}")
gc.collect()

--- Starting Block 2: Synthesized Feature Engineering ---
Creating brute-force numerical interaction features...
Creating TF-IDF features for text columns...
Finalizing feature set...

Synthesized FE complete. Total features: 111


10

In [3]:
# =============================================================================
# BLOCK 3: TWO-STAGE TUNING, TRAINING, AND SUBMISSION
# =============================================================================
from sklearn.metrics import mean_squared_error
print("\n--- Starting Block 3: Two-Stage Modeling Pipeline ---")

# --- STAGE 1: TUNE AND TRAIN THE MEAN MODEL ---
def objective_mean(trial):
    train_x, val_x, train_y, val_y = train_test_split(X, y_true, test_size=0.2, random_state=RANDOM_STATE)
    params = {
        'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'tree_method': 'hist',
        'eta': trial.suggest_float('eta', 0.01, 0.08, log=True),
        'max_depth': trial.suggest_int('max_depth', 6, 14),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
        'lambda': trial.suggest_float('lambda', 1e-4, 10.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-4, 10.0, log=True),
    }
    model = xgb.XGBRegressor(**params, n_estimators=1500, random_state=RANDOM_STATE, n_jobs=-1, early_stopping_rounds=50)
    model.fit(train_x, train_y, eval_set=[(val_x, val_y)], verbose=False)
    preds = model.predict(val_x)
    return np.sqrt(mean_squared_error(val_y, preds))

print("\n--- STAGE 1, PART 1: Tuning Mean Prediction Model ---")
study_mean = optuna.create_study(direction='minimize')
study_mean.optimize(objective_mean, n_trials=N_OPTUNA_TRIALS)
best_params_mean = study_mean.best_params
print(f"Mean Model Tuning Complete. Best Validation RMSE: ${study_mean.best_value:,.2f}")

print("\n--- STAGE 1, PART 2: K-Fold Training of Mean Model ---")
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
oof_mean_preds = np.zeros(len(X))
test_mean_preds = np.zeros(len(X_test))
for fold, (train_idx, val_idx) in enumerate(skf.split(X, df_train['grade'])):
    print(f"  Mean Model - Fold {fold+1}/{N_SPLITS}...")
    model = xgb.XGBRegressor(**best_params_mean, n_estimators=2000, objective='reg:squarederror', eval_metric='rmse', tree_method='hist', random_state=RANDOM_STATE, n_jobs=-1, early_stopping_rounds=100)
    model.fit(X.iloc[train_idx], y_true.iloc[train_idx], eval_set=[(X.iloc[val_idx], y_true.iloc[val_idx])], verbose=False)
    oof_mean_preds[val_idx] = model.predict(X.iloc[val_idx])
    test_mean_preds += model.predict(X_test) / N_SPLITS

# --- STAGE 2: TRAIN THE ERROR MODEL (No tuning for speed, using good defaults) ---
print("\n--- STAGE 2: K-Fold Training of Error Model ---")
error_target = np.abs(y_true - oof_mean_preds)
X_for_error = X.copy()
X_for_error['mean_pred_oof'] = oof_mean_preds
X_test_for_error = X_test.copy()
X_test_for_error['mean_pred_oof'] = test_mean_preds
params_error = {'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'tree_method': 'hist', 'eta': 0.03, 'max_depth': 7, 'random_state': RANDOM_STATE}
oof_error_preds = np.zeros(len(X))
test_error_preds = np.zeros(len(X_test))
for fold, (train_idx, val_idx) in enumerate(skf.split(X_for_error, df_train['grade'])):
    print(f"  Error Model - Fold {fold+1}/{N_SPLITS}...")
    model = xgb.XGBRegressor(**params_error, n_estimators=1500, n_jobs=-1, early_stopping_rounds=100)
    model.fit(X_for_error.iloc[train_idx], error_target.iloc[train_idx], eval_set=[(X_for_error.iloc[val_idx], error_target.iloc[val_idx])], verbose=False)
    oof_error_preds[val_idx] = model.predict(X_for_error.iloc[val_idx])
    test_error_preds += model.predict(X_test_for_error) / N_SPLITS

# --- FINAL ASYMMETRIC CALIBRATION AND SUBMISSION ---
print("\n--- Final Asymmetric Calibration ---")
oof_error_final = np.clip(oof_error_preds, 0, None) # Ensure error is not negative
best_a, best_b, best_metric = 2.0, 2.0, float('inf')
for a in np.arange(1.90, 2.31, 0.01):
    for b in np.arange(2.10, 2.51, 0.01):
        low = oof_mean_preds - oof_error_final * a
        high = oof_mean_preds + oof_error_final * b
        metric, coverage = winkler_score(y_true, low, high, alpha=COMPETITION_ALPHA, return_coverage=True)
        if metric < best_metric:
            best_metric = metric
            best_a, best_b = a, b
            print(f"New Best! a={best_a:.2f}, b={best_b:.2f}, Score={best_metric:,.2f}, Cov={coverage:.2%}")
print(f"\nGrid search complete. Final OOF Score: {best_metric:,.2f}. Best multipliers: a={best_a:.2f}, b={best_b:.2f}")



[I 2025-07-06 18:47:59,844] A new study created in memory with name: no-name-b04a7426-01fc-40d9-be0d-37229afb49b1



--- Starting Block 3: Two-Stage Modeling Pipeline ---

--- STAGE 1, PART 1: Tuning Mean Prediction Model ---


[I 2025-07-06 18:49:23,902] Trial 0 finished with value: 108788.92178893952 and parameters: {'eta': 0.06390775093802926, 'max_depth': 14, 'subsample': 0.9801996480974612, 'colsample_bytree': 0.8955994797575587, 'lambda': 0.0014640381248415897, 'alpha': 8.476133865702552}. Best is trial 0 with value: 108788.92178893952.
[I 2025-07-06 18:49:38,858] Trial 1 finished with value: 101225.98496433611 and parameters: {'eta': 0.04747122154060738, 'max_depth': 6, 'subsample': 0.9606143808140577, 'colsample_bytree': 0.8791188529638809, 'lambda': 0.00011325690288851288, 'alpha': 4.605557557546353}. Best is trial 1 with value: 101225.98496433611.
[I 2025-07-06 18:51:21,459] Trial 2 finished with value: 105649.01650275785 and parameters: {'eta': 0.023345559921624893, 'max_depth': 13, 'subsample': 0.9617487529268152, 'colsample_bytree': 0.8804904884526665, 'lambda': 0.17496762093974744, 'alpha': 0.14063427122774758}. Best is trial 1 with value: 101225.98496433611.
[I 2025-07-06 18:52:38,223] Trial 3 

Mean Model Tuning Complete. Best Validation RMSE: $99,711.33

--- STAGE 1, PART 2: K-Fold Training of Mean Model ---
  Mean Model - Fold 1/5...
  Mean Model - Fold 2/5...
  Mean Model - Fold 3/5...
  Mean Model - Fold 4/5...
  Mean Model - Fold 5/5...

--- STAGE 2: K-Fold Training of Error Model ---
  Error Model - Fold 1/5...
  Error Model - Fold 2/5...
  Error Model - Fold 3/5...
  Error Model - Fold 4/5...
  Error Model - Fold 5/5...

--- Final Asymmetric Calibration ---
New Best! a=1.90, b=2.10, Score=305,995.06, Cov=88.76%
New Best! a=1.90, b=2.11, Score=305,944.31, Cov=88.83%
New Best! a=1.90, b=2.12, Score=305,900.40, Cov=88.89%
New Best! a=1.90, b=2.13, Score=305,862.05, Cov=88.94%
New Best! a=1.90, b=2.14, Score=305,830.10, Cov=89.00%
New Best! a=1.90, b=2.15, Score=305,804.18, Cov=89.06%
New Best! a=1.90, b=2.16, Score=305,784.71, Cov=89.12%
New Best! a=1.90, b=2.17, Score=305,772.83, Cov=89.19%
New Best! a=1.90, b=2.18, Score=305,766.56, Cov=89.24%
New Best! a=1.91, b=2.14, 

,id,pi_lower,pi_upper
0,0,822391.902314,1.038016e+06
1,1,603598.815254,8.239731e+05
2,2,442467.346729,6.716598e+05
3,3,279592.376924,4.055703e+05
4,4,344854.317754,7.795774e+05


In [7]:
# =============================================================================
# FINAL ASYMMETRIC CALIBRATION AND SUBMISSION (CORRECTED)
# =============================================================================

print("\n--- Final Asymmetric Calibration ---")
oof_error_final = np.clip(oof_error_preds, 0, None) # Ensure error is not negative
best_a, best_b, best_metric = 2.0, 2.0, float('inf')

# This grid search finds the best multipliers based on your OOF predictions
for a in np.arange(1.90, 2.31, 0.01):
    for b in np.arange(2.10, 2.51, 0.01):
        low = oof_mean_preds - oof_error_final * a
        high = oof_mean_preds + oof_error_final * b
        metric, coverage = winkler_score(y_true, low, high, alpha=COMPETITION_ALPHA, return_coverage=True)
        if metric < best_metric:
            best_metric = metric
            best_a, best_b = a, b
            # This print is optional but good for seeing progress
            print(f"New Best! a={best_a:.2f}, b={best_b:.2f}, Score={best_metric:,.2f}, Cov={coverage:.2%}")

print(f"\nGrid search complete. Final OOF Score: {best_metric:,.2f}. Best multipliers: a={best_a:.2f}, b={best_b:.2f}")


# --- Create Final Submission ---
print("\nCreating final submission file...")
test_error_final = np.clip(test_error_preds, 0, None)
final_lower = test_mean_preds - test_error_final * best_a
final_upper = test_mean_preds + test_error_final * best_b
final_upper = np.maximum(final_lower, final_upper)

# --- THE FIX IS HERE ---
# Instead of using X_test.index, which was reset, we use the 'test_ids'
# variable that we saved in the feature engineering block.
test_ids = pd.read_csv('./test.csv', usecols=['id'])['id']
submission_df = pd.DataFrame({
    'id': test_ids, 
    'pi_lower': final_lower, 
    'pi_upper': final_upper
})

submission_df.to_csv('submission_ultimate_synthesis_final.csv', index=False)
print("\n'submission_ultimate_synthesis_final.csv' created successfully!")
display(submission_df.head())


--- Final Asymmetric Calibration ---
New Best! a=1.90, b=2.10, Score=305,995.06, Cov=88.76%
New Best! a=1.90, b=2.11, Score=305,944.31, Cov=88.83%
New Best! a=1.90, b=2.12, Score=305,900.40, Cov=88.89%
New Best! a=1.90, b=2.13, Score=305,862.05, Cov=88.94%
New Best! a=1.90, b=2.14, Score=305,830.10, Cov=89.00%
New Best! a=1.90, b=2.15, Score=305,804.18, Cov=89.06%
New Best! a=1.90, b=2.16, Score=305,784.71, Cov=89.12%
New Best! a=1.90, b=2.17, Score=305,772.83, Cov=89.19%
New Best! a=1.90, b=2.18, Score=305,766.56, Cov=89.24%
New Best! a=1.91, b=2.14, Score=305,752.56, Cov=89.08%
New Best! a=1.91, b=2.15, Score=305,726.64, Cov=89.14%
New Best! a=1.91, b=2.16, Score=305,707.17, Cov=89.21%
New Best! a=1.91, b=2.17, Score=305,695.29, Cov=89.27%
New Best! a=1.91, b=2.18, Score=305,689.02, Cov=89.32%
New Best! a=1.92, b=2.14, Score=305,683.81, Cov=89.17%
New Best! a=1.92, b=2.15, Score=305,657.89, Cov=89.23%
New Best! a=1.92, b=2.16, Score=305,638.42, Cov=89.29%
New Best! a=1.92, b=2.17, S

,id,pi_lower,pi_upper
0,200000,822391.902314,1.038016e+06
1,200001,603598.815254,8.239731e+05
2,200002,442467.346729,6.716598e+05
3,200003,279592.376924,4.055703e+05
4,200004,344854.317754,7.795774e+05
